## Importing the libraries

In [1]:
import pandas as pd
import math
from nltk.tokenize import word_tokenize
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

## Defining the documents (corpus) dictionary

In [2]:
corpus = {
    "doc_1": "Software engineering at Damascus university Software",
    "doc_2": "Information retrieval at Damascus university",
    "doc_3": "Indexing Information retrieval"
}

df = pd.DataFrame(corpus, index=["Document"])
df

,doc_1,doc_2,doc_3
Document,Software engineering at Damascus university So...,Information retrieval at Damascus university,Indexing Information retrieval


## Tokenizing & storing the indexing terms

In [3]:
indexing_terms = set()

for doc in corpus.values():
    # TODO : PROCESS THE TEXT BEFORE CREATING THE INDEXING TERMS: STEMMING, LEMMATIZATION, LOWER CASE, ..ETC
    indexing_terms.update(word_tokenize(doc))

indexing_terms_df = pd.DataFrame(indexing_terms)
indexing_terms_df

,0
0,Indexing
1,Information
2,Damascus
3,at
4,university
5,Software
6,retrieval
7,engineering


## Create the inverted index

In [4]:
inverted_index = defaultdict(list)

for docId, doc in corpus.items():
    doc_terms = set(word_tokenize(doc))
    for term in doc_terms:
        inverted_index[term].append(docId)

dict(inverted_index)

{'Damascus': ['doc_1', 'doc_2'],
 'at': ['doc_1', 'doc_2'],
 'university': ['doc_1', 'doc_2'],
 'Software': ['doc_1'],
 'engineering': ['doc_1'],
 'Information': ['doc_2', 'doc_3'],
 'retrieval': ['doc_2', 'doc_3'],
 'Indexing': ['doc_3']}

## Calculating the terms frequencies in a document

In [5]:
def calculate_tf(doc:str):
    tf = {}
    doc_terms = word_tokenize(doc)
    for term in doc_terms:
        tf[term] = (doc_terms.count(term) / len(doc_terms))
    return tf

tf_df = pd.DataFrame(calculate_tf(corpus['doc_1']), index=["tf"])
tf_df

,Software,engineering,at,Damascus,university
tf,0.333333,0.166667,0.166667,0.166667,0.166667


## Calculating the inverse document frequencies

In [6]:
def calculate_idf():
    idf = {}

    docs_count = len(corpus)

    for term, doc_ids in inverted_index.items():
        idf[term] = math.log((docs_count / len(doc_ids)) + 1)

    return idf


idf_df = pd.DataFrame(calculate_idf(), index=["idf"])
idf_df

,Damascus,at,university,Software,engineering,Information,retrieval,Indexing
idf,0.916291,0.916291,0.916291,1.386294,1.386294,0.916291,0.916291,1.386294


## Calculating tf-idf for the document

In [7]:
tf_idf = {}

doc_terms = word_tokenize(corpus['doc_1'])

doc_tf = calculate_tf(corpus['doc_1'])

idf = calculate_idf()

for term in doc_terms:
    tf_idf[term] = doc_tf[term] * idf[term]

tf_idf_df = pd.DataFrame(tf_idf, index=["tf_idf"])
tf_idf_df

,Software,engineering,at,Damascus,university
tf_idf,0.462098,0.231049,0.152715,0.152715,0.152715


## Calculate tf-idf using Scikit Learn TfidfVectorizer

In [8]:
documents = list(corpus.values())

# TODO : ADD YOUR OWN TOKENIZER & PREPROCESSOR !
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the documents
tfidf_matrix = vectorizer.fit_transform(documents)

df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=corpus.keys())

df

,at,damascus,engineering,indexing,information,retrieval,software,university
doc_1,0.293048,0.293048,0.385323,0.000000,0.000000,0.000000,0.770646,0.293048
doc_2,0.447214,0.447214,0.000000,0.000000,0.447214,0.447214,0.000000,0.447214
doc_3,0.000000,0.000000,0.000000,0.680919,0.517856,0.517856,0.000000,0.000000
